# Define set of halos we want to investigate further.

### Steps:<br/> Query for Halos: shows usage of CasJobs.<br/> Store them in HDF5 and CSV file : shows usage of MyScratch/File<br/> Store them in a table : shows usage of MyDB<br/> Share CSV file :  shows usage of SciDrive<br/> <!-- <img src="images/uc1.png"/> -->

In [ ]:
#  cleanup MyDB, MyScratchDB (NOT halo_particles_all), OwnCloud/cosmodata/ (NOT HaloParticles_All.h5), SciDrive/cosmodata

In [1]:
token="d68fcd6768b54c55ad1ec279a123421e"
import sys
sys.argv.append("--ident="+token)

In [2]:
# Import necessary libraries.
# "BOILERPLATE"
import SciServer.CasJobs
import SciServer.Keystone
import SciServer.SciDrive
import os
import pandas
import tables

In [3]:
# define SQL: query from Millennium halo merger trees tab 
# find halos with mass in given range
query = """SELECT haloId,np,x,y,z, halfMassRadius 
             FROM mpahalotrees..mr 
            WHERE snapnum=63 
              AND np between 200000 AND 210000"""

# query CasJobs table. Using MPAHaloTrees as context
queryResponse = SciServer.CasJobs.executeQuery(query, "MPAHaloTrees")

# parse results into py DataFrame for further in memory processing
halosDF = pandas.read_csv(queryResponse,index_col=0)
print("Found "+str(halosDF.count()[0])+" rows")

executeQuery POST response:  200 OK
Found 33 rows


In [11]:
# GOT  persistent/FileDB/cosmodata and make sure cosmdata is empty (apart from precooked file)

In [6]:
# store result as HDF5 file 
h5store = pandas.HDFStore('cosmodata/HaloSample.h5')
h5store['halos']=halosDF
h5store.close()

# store result as CSV file
halosDF.to_csv('cosmodata/HaloSample.csv')

In [5]:
# GOTO persistent/FileDB/cosmodata and see data has appeared

In [6]:
# GOTO CasJobs schema browser and ensure no table halo_sample

In [7]:
# when writing to MyDB, MUST have token
token = SciServer.Session.getKeystoneToken()

# create table in MyDB and write to it
ddl = """CREATE TABLE halo_sample(haloID bigint,np integer,x real,y real,z real, halfMassRadius real)"""
response = SciServer.CasJobs.executeQuery(ddl,token=token)

# upload directly form DataFrame
response=SciServer.CasJobs.uploadPandasDataFrameToTable(halosDF,"halo_sample",token=token)

executeQuery POST response:  200 OK
Uploading  2173 bytes...
uploadCVSDataFrameToTable POST response:  200 OK


In [ ]:
# GOTO CasJobs and see that table was written

In [8]:
# Make copy of CSV file in new container in SciDrive
filename = 'cosmodata/HaloSample.csv'
f = open(filename, 'rb')
data1 = f.read()
f.close()

In [ ]:
# GOTO SciDrive and show there is no container cosmodata

In [ ]:
# Make copy of CSV file in new container in SciDrive
filename = 'cosmodata/HaloSample.csv'
f = open(filename, 'rb')
data1 = f.read()
f.close()

# create container
container = 'cosmodata'
SciServer.SciDrive.createContainer(container)

# create file
scidrivename_name = container+"/HaloSample.csv"
SciServer.SciDrive.upload(scidrivename_name, data1)

In [ ]:
# GOTO SciDrive and see data was written